In [73]:
import pandas as pd
import io

proses ekstrak

In [ ]:
transaksi_df_raw = pd.read_csv('D:\Documents\sem5\gudat\gudang_data\DataTugasGudangData\transaksi.csv')
produk_df_raw = pd.read_excel('D:\Documents\sem5\gudat\gudang_data\DataTugasGudangData\produk.xlsx') 
feedback_df_raw = pd.read_csv('D:\Documents\sem5\gudat\gudang_data\DataTugasGudangData\feedback.txt')

In [75]:
print("\n[+] Data mentah 'transaksi.csv'")
print(transaksi_df_raw.head())
print("\n[+] Data mentah 'produk.xlsx'")
print(produk_df_raw.head())
print("\n[+] Data mentah feedback.txt'")
print(feedback_df_raw.head())


[+] Data mentah 'transaksi.csv'
  TransaksiID     Tanggal CustomerID ProdukID  Jumlah  HargaSatuan
0        T001  2025-08-01       C001     P001       2        50000
1        T002  2025-08-01       C002     P002       1        75000
2        T003  2025-08-02       C001     P003       3        20000
3        T004  2025-08-03       C003     P001       1        50000
4        T005  2025-08-03       C004     P002       2        75000

[+] Data mentah 'produk.xlsx'
  ProdukID                         NamaProduk    Kategori    Supplier
0     P001                      Laptop Lenovo  Elektronik   PT Sumber
1     P002                 Smartphone Samsung  Elektronik  PT IndoKom
2     P003  Buku Pengantar Pembelajaran Mesin  Pendidikan   SDU Press

[+] Data mentah feedback.txt'
  CustomerID                             Feedback Sentimen
0       C001         Pelayanan cepat, sangat puas  Positif
1       C002                  Harga terlalu mahal  Negatif
2       C003  Produk bagus tapi pengiriman lam

proses transformasi

1. standarisasi nama kolom

In [76]:
transaksi_df = transaksi_df_raw.rename(columns={
    'TransaksiID': 'transaction_id', 'Tanggal': 'date', 'CustomerID': 'customer_id',
    'ProdukID': 'product_id', 'Jumlah': 'quantity', 'HargaSatuan': 'price_per_unit'
})
produk_df = produk_df_raw.rename(columns={
    'ProdukID': 'product_id', 'NamaProduk': 'product_name', 'Kategori': 'kategori'
})
feedback_df = feedback_df_raw.rename(columns={
    'CustomerID': 'customer_id', 'Sentimen': 'sentimen'
})

2. transformasi tipe data dan hitung nilai baru

In [77]:
transaksi_df['date'] = pd.to_datetime(transaksi_df['date'])
transaksi_df['total_penjualan'] = transaksi_df['quantity'] * transaksi_df['price_per_unit']

3. membuat tabel dimensi (sebagai data master)

In [78]:
dim_produk = produk_df[['product_id', 'product_name', 'kategori']].copy()
dim_pelanggan = pd.merge(
    transaksi_df[['customer_id']].drop_duplicates(),
    feedback_df[['customer_id', 'sentimen']],
    on='customer_id', how='left'
)
dim_waktu = pd.DataFrame({'tanggal': transaksi_df['date'].unique()})
dim_waktu['tahun'] = dim_waktu['tanggal'].dt.year
dim_waktu['bulan'] = dim_waktu['tanggal'].dt.month
dim_waktu['hari'] = dim_waktu['tanggal'].dt.day

4. tabel fakta (untuk data transaksi)

In [79]:
fact_penjualan = transaksi_df[['transaction_id', 'customer_id', 'product_id', 'date', 'quantity', 'total_penjualan']].rename(columns={'date': 'tanggal'})

load dan analisis (menggabungkan dan analisis data)

In [80]:
df_analisis_penjualan = pd.merge(fact_penjualan, dim_produk, on='product_id')

In [81]:
total_penjualan_per_kategori = df_analisis_penjualan.groupby('kategori')['total_penjualan'].sum().reset_index()
total_penjualan_per_kategori = total_penjualan_per_kategori.sort_values(by='total_penjualan', ascending=False)
kepuasan_pelanggan = dim_pelanggan.dropna(subset=['sentimen'])

In [82]:
print("\n\nHASIL FINAL ")
print("\nJawaban 3a: Total Penjualan per Kategori Produk")
print(total_penjualan_per_kategori)
print("\nJawaban 3b: Data Kepuasan Pelanggan yang Tersedia")
print(kepuasan_pelanggan)



HASIL FINAL 

Jawaban 3a: Total Penjualan per Kategori Produk
     kategori  total_penjualan
0  Elektronik           375000
1  Pendidikan            60000

Jawaban 3b: Data Kepuasan Pelanggan yang Tersedia
  customer_id sentimen
0        C001  Positif
1        C002  Negatif
2        C003   Netral
3        C004  Negatif


simpan file final

In [83]:
with pd.ExcelWriter('gudang_data.xlsx') as writer:
    fact_penjualan.to_excel(writer, sheet_name='Fakta_Penjualan', index=False)
    dim_produk.to_excel(writer, sheet_name='Dimensi_Produk', index=False)
    dim_pelanggan.to_excel(writer, sheet_name='Dimensi_Pelanggan', index=False)
    dim_waktu.to_excel(writer, sheet_name='Dimensi_Waktu', index=False)

In [84]:
with pd.ExcelWriter('hasil_analisis.xlsx') as writer:
    total_penjualan_per_kategori.to_excel(writer, sheet_name='Penjualan_per_Kategori', index=False)
    kepuasan_pelanggan.to_excel(writer, sheet_name='Kepuasan_Pelanggan', index=False)

In [85]:
fact_penjualan.to_csv('fakta_penjualan.csv', index=False)
dim_produk.to_csv('dimensi_produk.csv', index=False)
total_penjualan_per_kategori.to_csv('hasil_penjualan_per_kategori.csv', index=False)